In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates import RandomLayers
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import time

## Classical hyperparameters

In [2]:
n_epochs = 30   # Number of optimization epochs
n_layers = 1    # Number of random layers
n_train = 50    # Size of the train dataset
n_test = 30      # Size of the test dataset
np.random.seed(0)           # Seed for NumPy random number generator
tf.random.set_seed(0)       # Seed for TensorFlow random number generator


# Hidden layers weights and bias
W1 = tf.Variable(tf.random.uniform([(28 // filter_size_v)* (28 // filter_size_h) * filters, out_h1], dtype = tf.float64))
b1 = tf.Variable(tf.random.uniform([out_h1,], dtype = tf.float64))
out_h1 = 10
# W2 = tf.Variable(tf.cast(tf.random.normal((out_h1, out_h2)), dtype = tf.float64))
# b2 = tf.Variable(tf.cast(tf.random.normal((out_h2,)), dtype = tf.float64))
# out_h2 = 10

## Quantum hyperparameters

In [ ]:
filter_size_h = 4
filter_size_v = 4
wires = filter_size_h * filter_size_v
filters = 1
layers = 1
params = tf.Variable(tf.random.uniform([filters,layers, wires], dtype = tf.float64))


    
#Here dev_amazon_braket
my_bucket = f"amazon-braket-Your-Bucket-Name" # the name of the bucket
my_prefix = "Your-Folder-Name" # the name of the folder in the bucket
s3_folder = (my_bucket, my_prefix)

device_arn = "arn:aws:braket:::device/quantum-simulator/amazon/sv1"

dev_amazon_braket_remote = qml.device(
  "braket.aws.qubit",
device_arn=device_arn,
wires=wires,
s3_destination_folder=s3_folder,
parallel=True,
)
dev_braket_local = qml.device("braket.local.qubit", wires=wires)
dev_pennylane_local = qml.device("default.qubit", wires=wires)
'''

## Load MNIST dataset

In [3]:
mnist_dataset = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist_dataset.load_data()

# Reduce dataset size
train_images = train_images[:n_train]
train_labels = tf.convert_to_tensor(train_labels[:n_train])
test_images = test_images[:n_test]
test_labels = tf.convert_to_tensor(test_labels[:n_test])

# Normalize pixel values within 0 and 1
train_images = train_images / 255
test_images = test_images / 255

# Add extra dimension for convolution channels
train_images = tf.convert_to_tensor(train_images[..., tf.newaxis])
test_images = tf.convert_to_tensor(test_images[..., tf.newaxis])

## Hybrid quantum-classical CCN model architecture

In [9]:
def quanv(image, params, filters = 1):

    if filter_size_h == 2:
        out = tf.zeros((14, 14, filters))
        # Loop over the coordinates of the top-left pixel of 2X2 squares
        I = []
        for i in range(filters):
            J = []
            for j in range(0, 28, 2):
                K = []
                for k in range(0, 28, 2):
                    # Process a squared 2x2 region of the image with a quantum circuit

                    q_results = quantum_scalar_prod(
                        tf.convert_to_tensor([
                            image[j, k, 0],
                            image[j, k + 1, 0],
                            image[j + 1, k, 0],
                            image[j + 1, k + 1, 0]
                        ]), params[i]
                    )
                    # Assign expectation values to different channels of the output pixel (j/2, k/2)
                    #relu
                    if q_results > 0:
                        K.append(q_results)
                    else:
                        K.append(0)
                J.append(K)
            I.append(J)
        return tf.convert_to_tensor(I)


    elif filter_size_h == 4:
        out = tf.zeros((7, 7, filters))
        # Loop over the coordinates of the top-left pixel of 2X2 squares
        I = []
        for i in range(filters):
            J = []
            for j in range(0, 28, 4):
                K = []
                for k in range(0, 28, 4):
                    # Process a squared 2x2 region of the image with a quantum circuit

                    q_results = quantum_scalar_prod(
                        tf.convert_to_tensor([
                            image[j, k, 0],
                            image[j, k + 1, 0],
                            image[j, k + 2, 0],
                            image[j, k + 3, 0],
                            image[j+1, k, 0],
                            image[j+1, k + 1, 0],
                            image[j+1, k + 2, 0],
                            image[j+1, k + 3, 0],
                            image[j+2, k, 0],
                            image[j+2, k + 1, 0],
                            image[j+2, k + 2, 0],
                            image[j+2, k + 3, 0],
                            image[j+3, k, 0],
                            image[j+3, k + 1, 0],
                            image[j+3, k + 2, 0],
                            image[j+3, k + 3, 0],


                        ]), params[i]
                    )
                    # Assign expectation values to different channels of the output pixel (j/2, k/2)
                    #relu
                    if q_results > 0:
                        K.append(q_results)
                    else:
                        K.append(0)
                J.append(K)
            I.append(J)
        return tf.convert_to_tensor(I)

    elif filter_size_h == 6:
        out = tf.zeros((6, 4, filters))
        # Loop over the coordinates of the top-left pixel of 2X2 squares
        I = []
        for i in range(filters):
            J = []
            for j in range(0, 28-6, 6):
                K = []
                for k in range(0, 28-5, 5):
                    # Process a squared 2x2 region of the image with a quantum circuit

                    #print(j,k)
                    q_results = quantum_scalar_prod(
                        tf.convert_to_tensor([

                            image[j, k, 0],
                            image[j, k + 1, 0],
                            image[j, k + 2, 0],
                            image[j, k + 3, 0],
                            image[j, k + 4, 0],

                            image[j+1, k, 0],
                            image[j+1, k + 1, 0],
                            image[j+1, k + 2, 0],
                            image[j+1, k + 3, 0],
                            image[j+1, k + 4, 0],

                            image[j+2, k, 0],
                            image[j+2, k + 1, 0],
                            image[j+2, k + 2, 0],
                            image[j+2, k + 3, 0],
                            image[j+2, k + 4, 0],

                            image[j+3, k, 0],
                            image[j+3, k + 1, 0],
                            image[j+3, k + 2, 0],
                            image[j+3, k + 3, 0],
                            image[j+3, k + 4, 0],

                            image[j+4, k, 0],
                            image[j+4, k + 1, 0],
                            image[j+4, k + 2, 0],
                            image[j+4, k + 3, 0],
                            image[j+4, k + 4, 0],

                            image[j+5, k, 0],
                            image[j+5, k + 1, 0],
                            image[j+5, k + 2, 0],
                            image[j+5, k + 3, 0],
                            image[j+5, k + 4, 0],


                        ]), params[i]
                    )
                    # Assign expectation values to different channels of the output pixel (j/2, k/2)
                    #relu
                    if q_results > 0:
                        K.append(q_results)
                    else:
                        K.append(0)
                J.append(K)
            I.append(J)
        return tf.convert_to_tensor(I)


def classical_scalar_prod(phi, params): #Classical

    sol = 0

    for i,j in zip(phi,params[0]):
        sol += i * j

    return sol

def quantum_scalar_prod(phi, params): ##Quantum 

    prob = circuit(phi,params)[0]
    return prob


@qml.qnode(dev_pennylane_local, interface = "tf")
def circuit(phi, params):

    # Encoding of 4 classical input values
    for j in range(wires):
        qml.RY(-np.pi * phi[j], wires=j)

    for layer in range(params.shape[0]):
        for i in range(params.shape[1]):
            qml.RX(np.pi * params[layer,i], wires = i)
            qml.CNOT(wires =[i, (i+1)%wires])
            qml.RY(np.pi * params[layer,i], wires = i)

    return qml.probs(wires = range(wires))


def x_hidden(x,W,b):
    # Applies a fully connected layer , i.e out = h(Wx+b) where h(·) is the softmax function
    x = tf.cast(x, dtype = tf.float64)
    #if last == False:
    #return tf.nn.relu(x @ W + b)

    sol = tf.nn.softmax(x @ W + b)
    #print("sol",sol)
    return sol

def to_vector(n):
    L = [0] * 10
    L[n] = 1
    return tf.cast(tf.convert_to_tensor(L), dtype = tf.float64)



### Training loop of a quantum convolutional filter layer + fully connected layer

In [ ]:
def accuracy():

    correct = 0
    for j in range(n_test):

        image = tf.reshape(quanv(test_images[j], params, filters),[1,-1])
        o1 = x_hidden(image,W1,b1)
        #o2 = x_hidden(o1,W2,b2, last = True)
       # o2=o1

        if tf.math.argmax(o1[0]) == tf.cast(test_labels[j], dtype = tf.int64):
            correct += 1
    return correct / n_test

losses = []
accuracies = []
start = time.time()
opt = keras.optimizers.SGD()
opt2 = keras.optimizers.Adadelta()
for i in range(n_epochs):
    epoch_loss = 0
    for j in range(n_train):
        with tf.GradientTape(persistent=True) as tape:
            image = tf.reshape(quanv(train_images[j], params, filters),[1,-1])
            o1 = x_hidden(image,W1,b1)
            #o2 = x_hidden(o1,W2,b2, last = True)
            label = tf.reshape(train_labels[j] , o1.shape[0])
            #loss = sum((o2 - label)[0] ** 2)
            #loss = tf.reduce_mean(-tf.reduce_sum(label * tf.math.log(o1)))
            loss = tf.keras.losses.sparse_categorical_crossentropy(label[0], o1) 
            # comprobar como esta definido el error, creo que algo falla
            epoch_loss += loss
            print(j," ",loss)
            print("Current Time(s) after loss plus previous gradient: ", time.time()-start)

        if j % 2 == 0:
            gradients = tape.gradient(loss, [W1,b1])
            #print("max grad", max(gradients[0][13]))
            opt.apply_gradients(zip(gradients, [W1, b1]))
        else:
            #print("random", W1[0][3])
            gradients = tape.gradient(loss, [params])
            opt.apply_gradients(zip(gradients, [params]))

    losses.append(epoch_loss)
    ac = accuracy()
    accuracies.append(ac)
    print("Epoch {}; loss: {} accuracy: {}".format(i,epoch_loss,ac))
    print("Current Time(s): ", time.time()-start)

In [8]:
import tensorflow as tf
tf.version.VERSION

'2.3.1'